# NASA Microbiome - PICRUSt 2

Run this notebook in `qiime2-2023.2`!

Continuing the [pd-mouse tutorial](https://docs.qiime2.org/2023.2/tutorials/pd-mice/). We'll discuss non-rarefaction based betadiversity and differential abundance. 

This notebook will focus on [PICRUSt 2](https://github.com/picrust/picrust2).

- [q2-picrust2](https://github.com/picrust/picrust2/wiki/q2-picrust2-Tutorial)
  - [Install PICRUS2 2 QIIME 2 plugin](https://library.qiime2.org/plugins/q2-picrust2/13/)
- [Main picrust2 Tutorial](https://github.com/picrust/picrust2/wiki/PICRUSt2-Tutorial-(v2.5.2))
  - [Stand-alone PICRUSt 2 environment install](https://github.com/picrust/picrust2/wiki/Installation).


In [15]:
from os import getcwd, listdir, chdir, mkdir
import qiime2 as q2

In [13]:
getcwd()

'/mnt/e/NASA_microbiome/processed/core-metrics-results/picrust2-output'

In [21]:
%cd /mnt/e/NASA_microbiome/

/mnt/e/NASA_microbiome


In [22]:
chdir('./processed')
getcwd()

'/mnt/e/NASA_microbiome/processed'

## Run full pipeline

There is debate on weather or not to use the rarefied or unrarefied table. Try both the rarefied and non-rarefied data. We'll use the rarefied data for now. Note we need to sync our sequence file with the rarefied table.



In [11]:
# filter seqs to match my rarefied table
! qiime feature-table filter-seqs \
    --i-data ./rep_set-no-ecmu.qza \
    --i-table ./core-metrics-results/rarefied_table.qza \
    --o-filtered-data ./core-metrics-results/rarefied_seqs.qza

Saved FeatureData[Sequence] to: ./core-metrics-results/rarefied_seqs.qza


In [13]:
! qiime picrust2 full-pipeline \
  --i-table ./core-metrics-results/rarefied_table.qza \
  --i-seq ./core-metrics-results/rarefied_seqs.qza \
  --p-threads 4 \
  --output-dir ./core-metrics-results/picrust2-output 

Saved FeatureTable[Frequency] to: ./core-metrics-results/picrust2-output/ko_metagenome.qza
Saved FeatureTable[Frequency] to: ./core-metrics-results/picrust2-output/ec_metagenome.qza
Saved FeatureTable[Frequency] to: ./core-metrics-results/picrust2-output/pathway_abundance.qza


In [14]:
! qiime feature-table summarize \
   --i-table ./core-metrics-results/picrust2-output/pathway_abundance.qza \
   --o-visualization ./core-metrics-results/picrust2-output/pathway_abundance.qzv

Saved Visualization to: ./core-metrics-results/picrust2-output/pathway_abundance.qzv


In [15]:
! qiime diversity core-metrics \
   --i-table ./core-metrics-results/picrust2-output/pathway_abundance.qza \
   --p-sampling-depth 102785 \
   --m-metadata-file NASA-Metadata.tsv \
   --output-dir ./core-metrics-results/picrust2-output/pathabund_core_metrics_out \
   --p-n-jobs 2

Saved FeatureTable[Frequency] to: ./core-metrics-results/picrust2-output/pathabund_core_metrics_out/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: ./core-metrics-results/picrust2-output/pathabund_core_metrics_out/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: ./core-metrics-results/picrust2-output/pathabund_core_metrics_out/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: ./core-metrics-results/picrust2-output/pathabund_core_metrics_out/evenness_vector.qza
Saved DistanceMatrix to: ./core-metrics-results/picrust2-output/pathabund_core_metrics_out/jaccard_distance_matrix.qza
Saved DistanceMatrix to: ./core-metrics-results/picrust2-output/pathabund_core_metrics_out/bray_curtis_distance_matrix.qza
Saved PCoAResults to: ./core-metrics-results/picrust2-output/pathabund_core_metrics_out/jaccard_pcoa_results.qza
Saved PCoAResults to: ./core-metrics-results/picrust2-output/pathabund_core_metrics_out/bray_curtis_pcoa_results.qza
Saved Visualization to: ./core-m

In [16]:
# Export files out of Qiime

In [27]:
! qiime tools export \
   --input-path ./core-metrics-results/picrust2-output/pathway_abundance.qza \
   --output-path pathabun_exported

Exported ./core-metrics-results/picrust2-output/pathway_abundance.qza as BIOMV210DirFmt to directory pathabun_exported


## Export/convert the .qza picrust2 output files to .biom files

In [40]:
!qiime tools export \
  --input-path ./core-metrics-results/picrust2-output/ko_metagenome.qza \
  --output-path ./core-metrics-results/picrust2-output/ko-table-biom 

Exported ./core-metrics-results/picrust2-output/ko_metagenome.qza as BIOMV210DirFmt to directory ./core-metrics-results/picrust2-output/ko-table-biom


In [41]:
!qiime tools export \
  --input-path ./core-metrics-results/picrust2-output/ec_metagenome.qza \
  --output-path ./core-metrics-results/picrust2-output/ec-table-biom 

Exported ./core-metrics-results/picrust2-output/ec_metagenome.qza as BIOMV210DirFmt to directory ./core-metrics-results/picrust2-output/ec-table-biom


In [42]:
!qiime tools export \
  --input-path ./core-metrics-results/picrust2-output/pathway_abundance.qza \
  --output-path ./core-metrics-results/picrust2-output/pathway-abundance-biom

Exported ./core-metrics-results/picrust2-output/pathway_abundance.qza as BIOMV210DirFmt to directory ./core-metrics-results/picrust2-output/pathway-abundance-biom


## Convert EC, KO and Pathway abundance biom files to .tsv

In [57]:
#convert EC
!biom convert -i ./core-metrics-results/picrust2-output/ec-table-biom/feature-table.biom  -o ./core-metrics-results/picrust2-output/ec_to_tsv/ec-table.tsv  --to-tsv

In [58]:
#convert KO
!biom convert -i ./core-metrics-results/picrust2-output/ko-table-biom/feature-table.biom  -o ./core-metrics-results/picrust2-output/ko_to_tsv/-table.tsv  --to-tsv

In [59]:
# convert pathway abundance
!biom convert -i ./core-metrics-results/picrust2-output/pathway-abundance-biom/feature-table.biom  -o ./core-metrics-results/picrust2-output/pathway-abundance-to_tsv/pathway-abundance-table.tsv  --to-tsv

In [60]:
!pwd

/mnt/e/NASA_microbiome/processed
